### 0. Imports

In [1]:
import pandas as pd

import requests
import json

from sklearn.metrics import mean_absolute_error

### 1. Fetch Data (EIA API v1 Interaction)

In [2]:
base_eia_url = 'http://api.eia.gov/series/'
key = '?api_key=1JB8lVq37ALJXVOd4UzkhNLkoyY8gDgAM7rsVIlC'

**PGE Hourly Demand and Day-ahead Forecast**

In [3]:
series1 = '&series_id=EBA.PGE-ALL.D.H'
response1 = requests.get(f'{base_eia_url}{key}{series1}')

pge_demand = pd.DataFrame(response1.json()['series'][0]['data'])

In [4]:
pge_demand

,0,1
0,20221010T17Z,2414.0
1,20221010T16Z,2414.0
2,20221010T15Z,2391.0
3,20221010T14Z,2227.0
4,20221010T13Z,1991.0
...,...,...
63078,20150722T12Z,1801.0
63079,20150722T11Z,1766.0
63080,20150722T10Z,1784.0
63081,20150722T09Z,1856.0


In [5]:
pge_demand.rename(columns={0: 'utc', 1: 'MWh'}, inplace=True)

In [6]:
pge_demand

,utc,MWh
0,20221010T17Z,2414.0
1,20221010T16Z,2414.0
2,20221010T15Z,2391.0
3,20221010T14Z,2227.0
4,20221010T13Z,1991.0
...,...,...
63078,20150722T12Z,1801.0
63079,20150722T11Z,1766.0
63080,20150722T10Z,1784.0
63081,20150722T09Z,1856.0


In [7]:
series2 = '&series_id=EBA.PGE-ALL.DF.H'
response2 = requests.get(f'{base_eia_url}{key}{series2}')

pge_forecast = pd.DataFrame(response2.json()['series'][0]['data'])

In [8]:
pge_forecast

,0,1
0,20221011T07Z,2046
1,20221011T06Z,2195
2,20221011T05Z,2370
3,20221011T04Z,2526
4,20221011T03Z,2680
...,...,...
61700,20150910T12Z,1737
61701,20150910T11Z,1704
61702,20150910T10Z,1728
61703,20150910T09Z,1789


In [9]:
pge_forecast.rename(columns={0: 'utc', 1: 'MWh'}, inplace=True)

In [10]:
pge_forecast

,utc,MWh
0,20221011T07Z,2046
1,20221011T06Z,2195
2,20221011T05Z,2370
3,20221011T04Z,2526
4,20221011T03Z,2680
...,...,...
61700,20150910T12Z,1737
61701,20150910T11Z,1704
61702,20150910T10Z,1728
61703,20150910T09Z,1789


### 2. Merge and Evaluate PGE's Model

In [11]:
pge_demand.rename(columns={'MWh': 'demand_MWh'}, inplace=True)

In [12]:
pge_forecast.rename(columns={'MWh': 'forecast_MWh'}, inplace=True)

In [13]:
pge = pge_demand.merge(pge_forecast, how='left')

In [14]:
pge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63083 entries, 0 to 63082
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   utc           63083 non-null  object 
 1   demand_MWh    62959 non-null  float64
 2   forecast_MWh  61691 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.9+ MB


In [15]:
pge[pge['demand_MWh'].isna()]

,utc,demand_MWh,forecast_MWh
25713,20191104T08Z,NaN,1989.0
25714,20191104T07Z,NaN,2147.0
25715,20191104T06Z,NaN,2327.0
25716,20191104T05Z,NaN,2461.0
25717,20191104T04Z,NaN,2556.0
...,...,...,...
35217,20180925T09Z,NaN,1640.0
35218,20180925T08Z,NaN,1711.0
49353,20170213T09Z,NaN,2045.0
51034,20161205T08Z,NaN,2286.0


In [16]:
pge[pge['forecast_MWh'].isna()]

,utc,demand_MWh,forecast_MWh
28426,20190714T07Z,2066.0,NaN
28427,20190714T06Z,2243.0,NaN
28428,20190714T05Z,2410.0,NaN
28429,20190714T04Z,2480.0,NaN
28430,20190714T03Z,2574.0,NaN
...,...,...,...
63078,20150722T12Z,1801.0,NaN
63079,20150722T11Z,1766.0,NaN
63080,20150722T10Z,1784.0,NaN
63081,20150722T09Z,1856.0,NaN


**For evaluating PGE's own forecasts, all rows with missing values dropped:**

In [17]:
pge_eval = pge.dropna()

In [18]:
pge_eval

,utc,demand_MWh,forecast_MWh
0,20221010T17Z,2414.0,2450.0
1,20221010T16Z,2414.0,2446.0
2,20221010T15Z,2391.0,2402.0
3,20221010T14Z,2227.0,2229.0
4,20221010T13Z,1991.0,1993.0
...,...,...,...
61878,20150910T12Z,1780.0,1737.0
61879,20150910T11Z,1745.0,1704.0
61880,20150910T10Z,1767.0,1728.0
61881,20150910T09Z,1835.0,1789.0


Note: No overlap between missing demand and forecast values.

In [19]:
mean_absolute_error(pge_eval['demand_MWh'], pge_eval['forecast_MWh'])

50.799844072311465

### 3. Save Data

In [20]:
pge_demand.to_csv('../data/pge_demand.csv', index=False)
pge_forecast.to_csv('../data/pge_forecast.csv', index=False)